In [6]:
import pandas as pd

folder = 'experiments/neg_dev_matched'

binary_finetune_pos_preds = pd.read_csv(folder + '/binary_finetune_pos_preds.tsv', header=None, delimiter='\t')
binary_finetune_pos_preds=binary_finetune_pos_preds.rename({0: "pos"}, axis=1)

binary_finetune_neg_preds = pd.read_csv(folder + '/binary_finetune_neg_preds.tsv', header=None, delimiter='\t')
binary_finetune_neg_preds=binary_finetune_neg_preds.rename({0: "neg"}, axis=1)

full_finetune_pos_preds = pd.read_csv(folder + '/full_two_way_finetune_pos_preds.tsv', header=None, delimiter='\t')
full_finetune_pos_preds=full_finetune_pos_preds.rename({0: "pos"}, axis=1)

full_finetune_neg_preds = pd.read_csv(folder + '/full_two_way_finetune_neg_preds.tsv', header=None, delimiter='\t')
full_finetune_neg_preds=full_finetune_neg_preds.rename({0: "neg"}, axis=1)

examples = pd.read_csv('neg_dev_matched.tsv', delimiter='\t')
binary_preds = pd.concat([binary_finetune_pos_preds, binary_finetune_neg_preds, examples], axis=1)
binary_preds['pos'] = binary_preds.apply(lambda row: 'c' if row['pos'] == 0 else 'e', axis=1)
binary_preds['neg'] = binary_preds.apply(lambda row: 'c' if row['neg'] == 0 else 'e', axis=1)
binary_preds['change'] = binary_preds.apply(lambda row: 1 if row['pos'] != row['neg'] else 0, axis=1)

full_preds = pd.concat([full_finetune_pos_preds, full_finetune_neg_preds, examples], axis=1)
full_preds['pos'] = full_preds.apply(lambda row: 'n' if row['pos'] == 0 else 'e', axis=1)
full_preds['neg'] = full_preds.apply(lambda row: 'n' if row['neg'] == 0 else 'e', axis=1)
full_preds['change'] = full_preds.apply(lambda row: 1 if row['pos'] != row['neg'] else 0, axis=1)

def combine_preds(row):
    return row['pos']+row['neg']

binary_preds['change_type'] = binary_preds.apply(combine_preds, axis=1)
full_preds['change_type'] = full_preds.apply(combine_preds, axis=1)


# display(binary_preds['change_type'].value_counts())
# display(full_preds['change_type'].value_counts())

print("percentage of initial examples: ")
print("binary: ")
total = len(binary_preds)
print(binary_preds['pos'].value_counts()/total)

print("full: ")
total = len(full_preds)
print(full_preds['pos'].value_counts()/total)


subset = full_preds[full_preds['pos']=="e"]
display(subset['neg'].value_counts()/len(subset))


percentage of initial examples: 
binary: 
e    0.528623
c    0.471377
Name: pos, dtype: float64
full: 
n    0.682227
e    0.317773
Name: pos, dtype: float64


n    0.654591
e    0.345409
Name: neg, dtype: float64

In [7]:
#first: pos logits
#second: neg logits
#after that: direct, indirect effects for each i

hidden_dim = 768
column_names = [None]*(2*hidden_dim+2)
column_names[0] = "pos_logits"
column_names[1] = "neg_logits"
for i in range(hidden_dim):
    column_names[i*2+2] = str(i) + "_dir"
    column_names[i*2+3] = str(i) + "_indir"

logits = pd.read_csv('experiments/may7/binary_finetune.tsv', header=None, delimiter='\t', names=column_names)

In [11]:
entailment_logits = logits.copy()
display(entailment_logits)

,pos_logits,neg_logits,0_dir,0_indir,1_dir,1_indir,2_dir,2_indir,3_dir,3_indir,...,763_dir,763_indir,764_dir,764_indir,765_dir,765_indir,766_dir,766_indir,767_dir,767_indir
0,"[0.8189356923103333, 0.18106429278850555]","[0.7719004154205322, 0.22809956967830658]","[0.7719480395317078, 0.22805191576480865]","[0.8188955783843994, 0.18110442161560059]","[0.7719001770019531, 0.22809989750385284]","[0.8189359307289124, 0.18106405436992645]","[0.7719041109085083, 0.2280958741903305]","[0.8189325928688049, 0.18106745183467865]","[0.7720416784286499, 0.22795835137367249]","[0.8188166618347168, 0.18118330836296082]",...,"[0.771982729434967, 0.22801733016967773]","[0.8188663721084595, 0.18113359808921814]","[0.7719653248786926, 0.22803469002246857]","[0.8188809752464294, 0.18111899495124817]","[0.7718992829322815, 0.2281007021665573]","[0.8189366459846497, 0.18106338381767273]","[0.7719295024871826, 0.22807052731513977]","[0.8189111948013306, 0.18108879029750824]","[0.7719926238059998, 0.22800730168819427]","[0.8188579678535461, 0.18114203214645386]"
1,"[0.898227334022522, 0.10177261382341385]","[0.9275789260864258, 0.0724211260676384]","[0.9275546669960022, 0.07244531065225601]","[0.898260235786438, 0.10173973441123962]","[0.9275789260864258, 0.0724211260676384]","[0.898227334022522, 0.10177261382341385]","[0.9275751113891602, 0.07242486625909805]","[0.8982325196266174, 0.10176751762628555]","[0.927501380443573, 0.07249864935874939]","[0.8983327746391296, 0.10166722536087036]",...,"[0.9275227785110474, 0.07247716188430786]","[0.8983036279678345, 0.1016964316368103]","[0.927540123462677, 0.07245989143848419]","[0.8982800245285034, 0.10171990096569061]","[0.9275792241096497, 0.07242083549499512]","[0.8982269763946533, 0.10177303105592728]","[0.9275604486465454, 0.07243954390287399]","[0.8982524275779724, 0.1017475575208664]","[0.9275371432304382, 0.07246282696723938]","[0.8982840776443481, 0.10171589255332947]"
2,"[0.9673073291778564, 0.03269265592098236]","[0.9730486869812012, 0.026951290667057037]","[0.973042368888855, 0.02695763297379017]","[0.9673150181770325, 0.03268501162528992]","[0.9730488061904907, 0.026951231062412262]","[0.9673073291778564, 0.03269273415207863]","[0.9730470180511475, 0.026952994987368584]","[0.9673094153404236, 0.03269060701131821]","[0.9730289578437805, 0.02697104588150978]","[0.9673311710357666, 0.03266885504126549]",...,"[0.9730322360992432, 0.026967739686369896]","[0.9673271775245667, 0.032672833651304245]","[0.973039448261261, 0.02696061320602894]","[0.967318594455719, 0.0326814241707325]","[0.9730488061904907, 0.02695116586983204]","[0.9673072099685669, 0.0326928049325943]","[0.9730437397956848, 0.02695632167160511]","[0.9673134684562683, 0.03268659487366676]","[0.9730383157730103, 0.02696164697408676]","[0.9673197865486145, 0.032680172473192215]"
3,"[0.9671698212623596, 0.03283024951815605]","[0.9813395142555237, 0.018660515546798706]","[0.9813249707221985, 0.01867503672838211]","[0.9671949148178101, 0.032805077731609344]","[0.9813395142555237, 0.018660519272089005]","[0.9671698212623596, 0.03283023461699486]","[0.9813387393951416, 0.0186612568795681]","[0.9671710133552551, 0.03282895311713219]","[0.9813007116317749, 0.018699323758482933]","[0.9672369360923767, 0.03276308998465538]",...,"[0.9813162088394165, 0.018683746457099915]","[0.9672099351882935, 0.03279000148177147]","[0.9813207983970642, 0.018679175525903702]","[0.9672021269798279, 0.032797910273075104]","[0.9813397526741028, 0.018660252913832664]","[0.9671692252159119, 0.032830700278282166]","[0.9813299775123596, 0.01866994984447956]","[0.9671860933303833, 0.03281388431787491]","[0.9813106060028076, 0.018689386546611786]","[0.9672197699546814, 0.03278025984764099]"
4,"[0.5252885818481445, 0.47471141815185547]","[0.5897467732429504, 0.4102531969547272]","[0.5896796584129333, 0.41032031178474426]","[0.5253577828407288, 0.47464224696159363]","[0.5897470116615295, 0.41025298833847046]","[0.5252884030342102, 0.47471165657043457]","[0.5897458791732788

In [23]:
for i, column in enumerate(logits.columns):
    entailment_logits[column] = logits[column].apply(lambda x: (x.split(",")[0][1:]))

In [26]:
entailment_logits = entailment_logits.astype('float')

IndexError: index 1537 is out of bounds for axis 0 with size 1537

In [ ]:
print(len(entailment_logits))

In [ ]:
#get average change for direct effects



#get average change for indirect effects

